
# Revenue Analytics Demo — KPI Notebook

This notebook computes **SaaS revenue KPIs** from synthetic data:
- **MRR / ARR**
- **Churn Rate**
- **Net Revenue Retention (NRR)**
- Breakdowns by **segment** and **region**

It uses the `customers.csv` and `revenue.csv` files generated earlier and mirrors a dbt-style fact model inside the notebook for convenience.

> Tip: Commit this notebook to your GitHub repo alongside the CSVs and the `models/fct_revenue.sql` file.


In [ ]:

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Never set custom styles or colors here to keep things portable
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)

DATA_DIR = Path('.')  # adjust if needed
CUSTOMERS_PATH = Path('/mnt/data/customers.csv')  # replace with 'data/customers.csv' in your repo
REVENUE_PATH = Path('/mnt/data/revenue.csv')      # replace with 'data/revenue.csv' in your repo
OUTPUTS_DIR = Path('/mnt/data/outputs')
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)


## 1. Load Datasets

In [ ]:

customers = pd.read_csv(CUSTOMERS_PATH, parse_dates=['start_date','churn_date'])
revenue = pd.read_csv(REVENUE_PATH, parse_dates=['month'])

print('Customers shape:', customers.shape)
display(customers.head())
print('Revenue shape:', revenue.shape)
display(revenue.head())


## 2. Build Month-Grained Fact Table (dbt-style)

In [ ]:

# Join
fct = (revenue
       .merge(customers, on='customer_id', how='left', suffixes=('','_cust'))
       .copy())

# Flags & fields
fct['is_churn_month'] = ((fct['churned'] == 1) & (fct['churn_date'].dt.to_period('M') == fct['month'].dt.to_period('M'))).astype(int)
fct['is_active'] = ((fct['month'] >= fct['start_date']) & ((fct['churn_date'].isna()) | (fct['month'] <= fct['churn_date']))).astype(int)

# prev_mrr by customer
fct = fct.sort_values(['customer_id','month'])
fct['prev_mrr'] = fct.groupby('customer_id')['mrr'].shift(1)
fct['mrr_delta'] = fct['mrr'] - fct['prev_mrr']

def classify(row):
    if row['is_churn_month'] == 1:
        return 'churn'
    if pd.notna(row['mrr_delta']):
        if row['mrr_delta'] > 0:
            return 'expansion'
        if row['mrr_delta'] < 0:
            return 'contraction'
    return 'flat'

fct['revenue_change_type'] = fct.apply(classify, axis=1)
fct['arr'] = fct['mrr'] * 12.0

display(fct.head(10))


## 3. KPI Calculations

In [ ]:

# Helper: month order
def month_order(df):
    return df.sort_values('month')['month'].unique()

# MRR by month (active only)
mrr_monthly = (fct[fct['is_active'] == 1]
               .groupby('month', as_index=False)
               .agg(mrr=('mrr','sum')))

# ARR by month
arr_monthly = (fct.groupby('month', as_index=False)
               .agg(arr=('arr','sum')))

# Churn MRR per month (sum of mrr where churn happened that month)
churn_mrr_monthly = (fct[fct['revenue_change_type'] == 'churn']
                     .groupby('month', as_index=False)
                     .agg(churn_mrr=('mrr','sum')))

# For churn rate, divide churn MRR by prior month's active MRR
mrr_prev = mrr_monthly.copy()
mrr_prev['mrr_prev'] = mrr_prev['mrr'].shift(1)
churn_base = (mrr_prev[['month','mrr_prev']]
              .merge(churn_mrr_monthly, on='month', how='left')
              .fillna({'churn_mrr':0.0}))
churn_base['churn_rate'] = np.where(churn_base['mrr_prev']>0, churn_base['churn_mrr']/churn_base['mrr_prev'], np.nan)
churn_rates = churn_base[['month','churn_rate']]

# Net Revenue Retention (NRR): sum(current MRR for existing customers) / sum(prev MRR for those same customers)
existing = fct[fct['prev_mrr'].notna()].copy()
nrr_monthly = (existing.groupby('month', as_index=False)
               .agg(current=('mrr','sum'), prev=('prev_mrr','sum')))
nrr_monthly['NRR'] = np.where(nrr_monthly['prev']>0, nrr_monthly['current']/nrr_monthly['prev'], np.nan)

# Segment & region breakdowns (ARR example)
arr_by_segment = (fct.groupby(['month','segment'], as_index=False)
                  .agg(arr=('arr','sum')))
arr_by_region = (fct.groupby(['month','region'], as_index=False)
                 .agg(arr=('arr','sum')))

display(mrr_monthly.head())
display(arr_monthly.head())
display(churn_rates.head())
display(nrr_monthly.head())


## 4. Quick Visuals (matplotlib)

In [ ]:

# 4.1 MRR trend
plt.figure()
plt.plot(mrr_monthly['month'], mrr_monthly['mrr'])
plt.title('Monthly Recurring Revenue (MRR)')
plt.xlabel('Month')
plt.ylabel('MRR (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'mrr_trend.png')
plt.show()

# 4.2 ARR trend
plt.figure()
plt.plot(arr_monthly['month'], arr_monthly['arr'])
plt.title('Annual Recurring Revenue (ARR)')
plt.xlabel('Month')
plt.ylabel('ARR (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'arr_trend.png')
plt.show()

# 4.3 Churn Rate trend
plt.figure()
plt.plot(churn_rates['month'], churn_rates['churn_rate'])
plt.title('Churn Rate (MRR churn / prior active MRR)')
plt.xlabel('Month')
plt.ylabel('Churn Rate')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'churn_rate_trend.png')
plt.show()

# 4.4 NRR trend
plt.figure()
plt.plot(nrr_monthly['month'], nrr_monthly['NRR'])
plt.title('Net Revenue Retention (NRR)')
plt.xlabel('Month')
plt.ylabel('NRR')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUTPUTS_DIR / 'nrr_trend.png')
plt.show()


## 5. Export KPI Tables & Figures

In [ ]:

mrr_monthly.to_csv(OUTPUTS_DIR / 'mrr_monthly.csv', index=False)
arr_monthly.to_csv(OUTPUTS_DIR / 'arr_monthly.csv', index=False)
churn_rates.to_csv(OUTPUTS_DIR / 'churn_rates.csv', index=False)
nrr_monthly.to_csv(OUTPUTS_DIR / 'nrr_monthly.csv', index=False)

print('Saved CSVs to:', OUTPUTS_DIR)
print('Saved figures: mrr_trend.png, arr_trend.png, churn_rate_trend.png, nrr_trend.png')



## 6. Next Steps

- Add **forecasting** (Prophet or ARIMA) using `mrr_monthly` as input.
- Compute **segment-level NRR** and **logo churn vs. revenue churn**.
- Publish selected CSVs to a `data/` folder in your repo and wire into **Power BI**.
- Commit the `outputs/` charts as static images for your README.
